# 🏥 Azure Application Insights - System Health & Performance Dashboard

This notebook creates a comprehensive, interactive dashboard that displays the complete overall system health, performance metrics, and failure analysis from Azure Application Insights using Kusto Query Language (KQL).

**Dashboard Features:**
- 📊 Overall Health Status Score
- ⚡ Performance Metrics (Response Times, Throughput)
- ❌ Failure Analysis (Error Rates, Exceptions)
- 📈 Trend Analysis
- 🎯 Key Performance Indicators (KPIs)
- 🔴🟡🟢 Color-coded Status Indicators

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta
import json
from azure.identity import DefaultAzureCredential
from azure.monitor.query import LogsQueryClient
import warnings

warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

## 2. Connect to Azure Application Insights

Set your Azure credentials and Application Insights details below:

In [ ]:
# Azure Configuration
# Replace these with your actual Application Insights workspace ID
WORKSPACE_ID = "your-workspace-id"  # e.g., "12345678-1234-1234-1234-123456789012"

# Initialize Azure credentials (uses default authentication chain)
try:
    credential = DefaultAzureCredential()
    client = LogsQueryClient(credential)
    print("✅ Azure connection established successfully!")
except Exception as e:
    print(f"⚠️  Note: To use live data, configure WORKSPACE_ID and Azure credentials.")
    print(f"    Error: {str(e)}")
    print("    For now, we'll use sample data for demonstration.")
    client = None

# Time range for query
end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=24)  # Last 24 hours

print(f"📅 Query time range: {start_time} to {end_time}")

## 3. Query Application Insights Data with KQL

Execute comprehensive KQL queries to retrieve system health, performance, and failure metrics:

In [ ]:
def execute_kql_query(kql_query):
    """Execute KQL query and return results as DataFrame"""
    if client is None:
        return None
    try:
        result = client.query_workspace(WORKSPACE_ID, kql_query)
        df = pd.DataFrame(result.tables[0].rows, columns=[col.name for col in result.tables[0].columns])
        return df
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# KQL Query 1: Overall System Health Metrics
kql_health = """
requests
| where timestamp > ago(24h)
| summarize
    TotalRequests = count(),
    SuccessfulRequests = countif(success == true),
    FailedRequests = countif(success == false),
    AverageResponseTime = avg(duration),
    P95ResponseTime = percentile(duration, 95),
    P99ResponseTime = percentile(duration, 99)
| extend
    SuccessRate = (SuccessfulRequests * 100.0) / TotalRequests,
    FailureRate = (FailedRequests * 100.0) / TotalRequests
"""

# KQL Query 2: Exception and Error Analysis
kql_exceptions = """
exceptions
| where timestamp > ago(24h)
| summarize 
    TotalExceptions = count(),
    CriticalCount = countif(severityLevel == 1),
    WarningCount = countif(severityLevel == 2),
    InfoCount = countif(severityLevel == 3)
| extend
    CriticalPercentage = (CriticalCount * 100.0) / TotalExceptions,
    WarningPercentage = (WarningCount * 100.0) / TotalExceptions
"""

# KQL Query 3: Performance by Operation
kql_performance = """
requests
| where timestamp > ago(24h)
| summarize
    RequestCount = count(),
    AvgDuration = avg(duration),
    MaxDuration = max(duration),
    SuccessCount = countif(success == true)
    by name
| extend
    SuccessRate = (SuccessCount * 100.0) / RequestCount
| order by AvgDuration desc
| limit 10
"""

# KQL Query 4: Availability and Uptime
kql_availability = """
availabilityResults
| where timestamp > ago(24h)
| summarize
    TestCount = count(),
    PassedTests = countif(success == true),
    FailedTests = countif(success == false)
| extend
    AvailabilityPercentage = (PassedTests * 100.0) / TestCount
"""

# KQL Query 5: Request Timeline (Hourly Trend)
kql_timeline = """
requests
| where timestamp > ago(24h)
| summarize
    RequestCount = count(),
    SuccessCount = countif(success == true),
    FailureCount = countif(success == false),
    AvgResponse = avg(duration)
    by bin(timestamp, 1h)
| order by timestamp asc
"""

print("📊 KQL queries defined. Attempting to retrieve data...\n")

In [ ]:
# Execute queries (or generate sample data if offline)
if client is not None:
    df_health = execute_kql_query(kql_health)
    df_exceptions = execute_kql_query(kql_exceptions)
    df_performance = execute_kql_query(kql_performance)
    df_availability = execute_kql_query(kql_availability)
    df_timeline = execute_kql_query(kql_timeline)
else:
    # Generate realistic sample data for demonstration
    print("📊 Generating sample data for visualization...\n")
    
    # Sample Health Data
    df_health = pd.DataFrame({
        'TotalRequests': [15847],
        'SuccessfulRequests': [15421],
        'FailedRequests': [426],
        'AverageResponseTime': [234.5],
        'P95ResponseTime': [892.3],
        'P99ResponseTime': [1245.7],
        'SuccessRate': [97.31],
        'FailureRate': [2.69]
    })
    
    # Sample Exception Data
    df_exceptions = pd.DataFrame({
        'TotalExceptions': [148],
        'CriticalCount': [12],
        'WarningCount': [45],
        'InfoCount': [91],
        'CriticalPercentage': [8.11],
        'WarningPercentage': [30.41]
    })
    
    # Sample Performance Data
    df_performance = pd.DataFrame({
        'name': ['API/Orders/GetDetails', 'API/Users/Authenticate', 'API/Reports/Generate', 
                 'API/Data/Sync', 'API/Files/Upload', 'API/Dashboard/Load'],
        'RequestCount': [2341, 3245, 1823, 4521, 1234, 2683],
        'AvgDuration': [542.3, 234.1, 1823.5, 456.7, 2134.2, 789.3],
        'MaxDuration': [3245.7, 1234.2, 8923.1, 2341.5, 9234.6, 4123.2],
        'SuccessCount': [2312, 3198, 1745, 4423, 1167, 2589],
        'SuccessRate': [98.76, 98.55, 95.73, 97.83, 94.56, 96.50]
    })
    
    # Sample Availability Data
    df_availability = pd.DataFrame({
        'TestCount': [144],
        'PassedTests': [142],
        'FailedTests': [2],
        'AvailabilityPercentage': [98.61]
    })
    
    # Sample Timeline Data
    hours = pd.date_range(end=datetime.utcnow(), periods=24, freq='H')
    df_timeline = pd.DataFrame({
        'timestamp': hours,
        'RequestCount': np.random.randint(500, 1200, 24),
        'SuccessCount': np.random.randint(480, 1180, 24),
        'FailureCount': np.random.randint(10, 100, 24),
        'AvgResponse': np.random.uniform(200, 400, 24)
    })

print("✅ Data retrieved successfully!")

## 4. Process and Aggregate Metrics

Calculate comprehensive health scores and performance indicators:

In [ ]:
def calculate_health_score(success_rate, response_time, availability, exceptions):
    """
    Calculate overall health score (0-100)
    Weighted factors:
    - Success Rate: 40%
    - Response Time: 30%
    - Availability: 20%
    - Exceptions: 10%
    """
    # Normalize success rate (already a percentage, max 100)
    success_score = min(success_rate, 100)
    
    # Normalize response time (target < 500ms, worst case > 2000ms)
    response_score = max(0, 100 - ((response_time - 200) / 18))
    response_score = min(100, max(0, response_score))
    
    # Normalize availability (already a percentage)
    availability_score = min(availability, 100)
    
    # Normalize exceptions (fewer is better)
    exception_score = max(0, 100 - (exceptions * 2))
    
    # Calculate weighted health score
    health_score = (
        (success_score * 0.40) +
        (response_score * 0.30) +
        (availability_score * 0.20) +
        (exception_score * 0.10)
    )
    
    return min(100, max(0, health_score))

def get_health_status(score):
    """Determine health status based on score"""
    if score >= 95:
        return "🟢 EXCELLENT", "green"
    elif score >= 85:
        return "🟢 HEALTHY", "lightgreen"
    elif score >= 70:
        return "🟡 WARNING", "yellow"
    elif score >= 50:
        return "🟠 CONCERNING", "orange"
    else:
        return "🔴 CRITICAL", "red"

# Extract metrics from dataframes
health_data = df_health.iloc[0]
exc_data = df_exceptions.iloc[0]
avail_data = df_availability.iloc[0]

success_rate = health_data['SuccessRate']
avg_response = health_data['AverageResponseTime']
availability = avail_data['AvailabilityPercentage']
total_exceptions = exc_data['TotalExceptions']

# Calculate overall health score
overall_health_score = calculate_health_score(
    success_rate,
    avg_response,
    availability,
    total_exceptions / 100  # Normalize exception count
)

health_status, health_color = get_health_status(overall_health_score)

# Performance metrics
p95_response = health_data['P95ResponseTime']
p99_response = health_data['P99ResponseTime']
throughput = health_data['TotalRequests'] / 24  # requests per hour

# Failure metrics
failure_rate = health_data['FailureRate']
critical_exceptions = exc_data['CriticalCount']
warning_exceptions = exc_data['WarningCount']

print(f"\n{'='*60}")
print(f"  OVERALL SYSTEM HEALTH SCORE: {overall_health_score:.1f}/100 {health_status}")
print(f"{'='*60}")
print(f"\n📊 KEY PERFORMANCE INDICATORS:")
print(f"  • Success Rate: {success_rate:.2f}%")
print(f"  • Average Response Time: {avg_response:.1f}ms")
print(f"  • P95 Response Time: {p95_response:.1f}ms")
print(f"  • P99 Response Time: {p99_response:.1f}ms")
print(f"  • Availability: {availability:.2f}%")
print(f"  • Throughput: {throughput:.1f} req/hour")
print(f"\n⚠️  FAILURE & ERROR METRICS:")
print(f"  • Failure Rate: {failure_rate:.2f}%")
print(f"  • Total Exceptions: {int(total_exceptions)}")
print(f"  • Critical Exceptions: {int(critical_exceptions)}")
print(f"  • Warning Exceptions: {int(warning_exceptions)}")
print(f"{'='*60}\n")

## 5. Create Comprehensive Health Dashboard Visualization

Build an interactive, multi-component dashboard:

In [ ]:
# Create comprehensive dashboard with multiple subplots
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=(
        "🏥 Overall Health Status",
        "⚡ Success vs Failure Rate",
        "🎯 Service Availability",
        "📈 Response Time Distribution",
        "🔴 Exception Severity Breakdown",
        "⏱️ Response Time Percentiles",
        "📊 Request Timeline (24h)",
        "🚀 Top Endpoints Performance",
        "⚙️ System Metrics Overview"
    ),
    specs=[
        [{"type": "indicator"}, {"type": "pie"}, {"type": "indicator"}],
        [{"type": "box"}, {"type": "pie"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "bar"}, {"type": "table"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
    row_heights=[0.25, 0.35, 0.40]
)

# 1. Overall Health Status (Gauge Chart)
fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=overall_health_score,
        title={"text": "Health Score"},
        domain={"x": [0, 1], "y": [0, 1]},
        gauge={
            "axis": {"range": [0, 100]},
            "bar": {"color": health_color},
            "steps": [
                {"range": [0, 50], "color": "rgba(255, 0, 0, 0.1)"},
                {"range": [50, 70], "color": "rgba(255, 165, 0, 0.1)"},
                {"range": [70, 85], "color": "rgba(255, 255, 0, 0.1)"},
                {"range": [85, 95], "color": "rgba(144, 238, 144, 0.1)"},
                {"range": [95, 100], "color": "rgba(0, 128, 0, 0.1)"}
            ],
            "threshold": {
                "line": {"color": "red", "width": 4},
                "thickness": 0.75,
                "value": 50
            }
        },
        delta={"reference": 85}
    ),
    row=1, col=1
)

# 2. Success vs Failure Rate (Pie Chart)
fig.add_trace(
    go.Pie(
        labels=["✅ Successful", "❌ Failed"],
        values=[success_rate, failure_rate],
        marker={"colors": ["#2ecc71", "#e74c3c"]},
        hovertemplate="<b>%{label}</b><br>%{value:.2f}%<extra></extra>",
        textposition="inside",
        textinfo="label+percent"
    ),
    row=1, col=2
)

# 3. Service Availability (Indicator)
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=availability,
        title={"text": "Availability %"},
        domain={"x": [0, 1], "y": [0, 1]},
        gauge={
            "axis": {"range": [95, 100]},
            "bar": {"color": "#3498db"},
            "steps": [
                {"range": [95, 97], "color": "rgba(52, 152, 219, 0.1)"},
                {"range": [97, 99], "color": "rgba(52, 152, 219, 0.2)"},
                {"range": [99, 100], "color": "rgba(52, 152, 219, 0.3)"}
            ]
        }
    ),
    row=1, col=3
)

# 4. Response Time Distribution (Box Plot)
response_times = [avg_response, p95_response, p99_response]
fig.add_trace(
    go.Box(y=response_times, name="Response Time (ms)", 
           marker={"color": "#9b59b6"}, boxmean='sd'),
    row=2, col=1
)

# 5. Exception Severity (Pie Chart)
exception_colors = ["#c0392b", "#f39c12", "#3498db"]
fig.add_trace(
    go.Pie(
        labels=["🔴 Critical", "🟡 Warning", "ℹ️ Info"],
        values=[critical_exceptions, warning_exceptions, exc_data['InfoCount']],
        marker={"colors": exception_colors},
        hovertemplate="<b>%{label}</b><br>Count: %{value}<extra></extra>",
        textinfo="label+value"
    ),
    row=2, col=2
)

# 6. Response Time Percentiles (Bar Chart)
percentile_labels = ["Avg", "P95", "P99"]
percentile_values = [avg_response, p95_response, p99_response]
fig.add_trace(
    go.Bar(
        x=percentile_labels,
        y=percentile_values,
        marker={"color": ["#2ecc71", "#f39c12", "#e74c3c"]},
        text=[f"{v:.0f}ms" for v in percentile_values],
        textposition="outside",
        hovertemplate="<b>%{x}</b><br>%{y:.1f}ms<extra></extra>",
        name="Response Time"
    ),
    row=2, col=3
)

# 7. Request Timeline (Scatter/Line Chart)
fig.add_trace(
    go.Scatter(
        x=df_timeline['timestamp'],
        y=df_timeline['RequestCount'],
        mode='lines+markers',
        name='Requests',
        line={"color": "#3498db", "width": 3},
        marker={"size": 6},
        hovertemplate="<b>%{x}</b><br>Requests: %{y}<extra></extra>",
        fill='tozeroy'
    ),
    row=3, col=1
)

# 8. Top Endpoints Performance (Horizontal Bar Chart)
top_endpoints = df_performance.nlargest(6, 'RequestCount')
fig.add_trace(
    go.Bar(
        y=top_endpoints['name'],
        x=top_endpoints['AvgDuration'],
        orientation='h',
        marker={"color": top_endpoints['SuccessRate'], 
                "colorscale": "RdYlGn", "showscale": False},
        text=[f"{v:.0f}ms" for v in top_endpoints['AvgDuration']],
        textposition="outside",
        hovertemplate="<b>%{y}</b><br>Avg: %{x:.0f}ms<extra></extra>",
        name="Avg Duration"
    ),
    row=3, col=2
)

# 9. System Metrics Summary Table
table_data = {
    "Metric": [
        "Total Requests",
        "Success Rate",
        "Failure Rate",
        "Avg Response",
        "P95 Response",
        "Availability",
        "Throughput",
        "Exceptions"
    ],
    "Value": [
        f"{int(health_data['TotalRequests']):,}",
        f"{success_rate:.2f}%",
        f"{failure_rate:.2f}%",
        f"{avg_response:.1f}ms",
        f"{p95_response:.1f}ms",
        f"{availability:.2f}%",
        f"{throughput:.1f}/hr",
        f"{int(total_exceptions)}"
    ],
    "Status": [
        "✅" if health_data['TotalRequests'] > 10000 else "⚠️",
        "✅" if success_rate > 95 else "⚠️",
        "✅" if failure_rate < 5 else "⚠️",
        "✅" if avg_response < 300 else "⚠️",
        "✅" if p95_response < 1000 else "⚠️",
        "✅" if availability > 99 else "⚠️",
        "✅" if throughput > 500 else "⚠️",
        "✅" if total_exceptions < 100 else "⚠️"
    ]
}

fig.add_trace(
    go.Table(
        header=dict(
            values=["<b>Metric</b>", "<b>Value</b>", "<b>Status</b>"],
            fill_color="#2c3e50",
            font=dict(color="white", size=11),
            height=25
        ),
        cells=dict(
            values=[table_data["Metric"], table_data["Value"], table_data["Status"]],
            fill_color=["#ecf0f1", "#ecf0f1"],
            font=dict(size=10),
            height=25
        )
    ),
    row=3, col=3
)

# Update layout
fig.update_layout(
    title={
        "text": "<b>🏥 Azure Application Insights - System Health & Performance Dashboard</b>",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 24, "color": "#2c3e50"}
    },
    showlegend=False,
    height=1400,
    width=1600,
    template="plotly_white",
    font={"family": "Arial, sans-serif", "size": 10},
    paper_bgcolor="#f8f9fa",
    plot_bgcolor="#ffffff",
    margin={"l": 50, "r": 50, "t": 100, "b": 50}
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')

# Display the dashboard
fig.show()

print("\n✅ Dashboard created successfully!")

## 6. Advanced Analytics - Deep Dive Analysis

In [ ]:
# Create detailed performance breakdown dashboard
fig2 = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "🚀 Endpoint Performance Comparison",
        "📊 Request Distribution",
        "⏱️ Response Time by Endpoint",
        "✅ Success Rate Ranking"
    ),
    specs=[
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "scatter"}, {"type": "bar"}]
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.12
)

# 1. Endpoint Requests Count
fig2.add_trace(
    go.Bar(
        x=df_performance['name'],
        y=df_performance['RequestCount'],
        marker={"color": "#3498db"},
        text=df_performance['RequestCount'],
        textposition="outside",
        hovertemplate="<b>%{x}</b><br>Requests: %{y:,.0f}<extra></extra>",
        name="Request Count"
    ),
    row=1, col=1
)

# 2. Success Count vs Failed
df_performance['FailedCount'] = df_performance['RequestCount'] - df_performance['SuccessCount']
fig2.add_trace(
    go.Bar(
        x=df_performance['name'],
        y=df_performance['SuccessCount'],
        marker={"color": "#2ecc71"},
        name="✅ Success",
        hovertemplate="<b>%{x}</b><br>Success: %{y:,.0f}<extra></extra>"
    ),
    row=1, col=2
)
fig2.add_trace(
    go.Bar(
        x=df_performance['name'],
        y=df_performance['FailedCount'],
        marker={"color": "#e74c3c"},
        name="❌ Failed",
        hovertemplate="<b>%{x}</b><br>Failed: %{y:,.0f}<extra></extra>"
    ),
    row=1, col=2
)

# 3. Response Time Trend
fig2.add_trace(
    go.Scatter(
        x=df_performance['name'],
        y=df_performance['AvgDuration'],
        mode='lines+markers',
        name='Avg Duration',
        line={"color": "#f39c12", "width": 3},
        marker={"size": 10},
        fill='tozeroy',
        hovertemplate="<b>%{x}</b><br>Avg: %{y:.0f}ms<extra></extra>"
    ),
    row=2, col=1
)
fig2.add_trace(
    go.Scatter(
        x=df_performance['name'],
        y=df_performance['MaxDuration'],
        mode='markers',
        name='Max Duration',
        marker={"size": 8, "color": "#e74c3c"},
        hovertemplate="<b>%{x}</b><br>Max: %{y:.0f}ms<extra></extra>"
    ),
    row=2, col=1
)

# 4. Success Rate Ranking
colors_success = ['#2ecc71' if x >= 95 else '#f39c12' if x >= 90 else '#e74c3c' 
                  for x in df_performance['SuccessRate']]
fig2.add_trace(
    go.Bar(
        x=df_performance['SuccessRate'],
        y=df_performance['name'],
        orientation='h',
        marker={"color": colors_success},
        text=[f"{x:.1f}%" for x in df_performance['SuccessRate']],
        textposition="outside",
        hovertemplate="<b>%{y}</b><br>Success: %{x:.2f}%<extra></extra>",
        name="Success Rate"
    ),
    row=2, col=2
)

fig2.update_layout(
    title={
        "text": "<b>📊 Detailed Endpoint Performance Analysis</b>",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22, "color": "#2c3e50"}
    },
    showlegend=True,
    height=900,
    width=1600,
    template="plotly_white",
    font={"family": "Arial, sans-serif", "size": 10},
    paper_bgcolor="#f8f9fa",
    plot_bgcolor="#ffffff",
    barmode='stack'
)

fig2.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')
fig2.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')

fig2.show()

print("✅ Detailed performance analysis dashboard created!")

## 7. Health Trend Analysis Over Time

In [ ]:
# Calculate hourly health scores for trend analysis
df_timeline['SuccessRate'] = (df_timeline['SuccessCount'] / df_timeline['RequestCount'] * 100).round(2)
df_timeline['FailureRate'] = (df_timeline['FailureCount'] / df_timeline['RequestCount'] * 100).round(2)
df_timeline['HourlyHealthScore'] = df_timeline.apply(
    lambda row: calculate_health_score(
        row['SuccessRate'],
        row['AvgResponse'],
        98.5,  # Average availability
        row['FailureCount'] / 10
    ),
    axis=1
)

# Create trend analysis dashboard
fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=("📈 System Health Score Trend (24h)", "📊 Requests & Errors Timeline"),
    specs=[[{"secondary_y": False}], [{"secondary_y": True}]],
    vertical_spacing=0.15
)

# 1. Health Score Trend
fig3.add_trace(
    go.Scatter(
        x=df_timeline['timestamp'],
        y=df_timeline['HourlyHealthScore'],
        mode='lines+markers',
        name='Health Score',
        line={"color": "#3498db", "width": 4},
        marker={"size": 8},
        fill='tozeroy',
        fillcolor='rgba(52, 152, 219, 0.2)',
        hovertemplate="<b>%{x|%H:%M}</b><br>Health: %{y:.1f}/100<extra></extra>"
    ),
    row=1, col=1
)

# Add a reference line for good health
fig3.add_hline(y=85, line_dash="dash", line_color="#f39c12", 
               annotation_text="Target: 85", row=1, col=1)

# 2. Requests Timeline
fig3.add_trace(
    go.Scatter(
        x=df_timeline['timestamp'],
        y=df_timeline['RequestCount'],
        mode='lines',
        name='Total Requests',
        line={"color": "#2ecc71", "width": 3},
        fill='tozeroy',
        fillcolor='rgba(46, 204, 113, 0.2)',
        hovertemplate="<b>%{x|%H:%M}</b><br>Requests: %{y:,.0f}<extra></extra>"
    ),
    row=2, col=1
)

# 3. Error Count (Secondary Axis)
fig3.add_trace(
    go.Scatter(
        x=df_timeline['timestamp'],
        y=df_timeline['FailureCount'],
        mode='lines+markers',
        name='Failures',
        line={"color": "#e74c3c", "width": 3},
        marker={"size": 6},
        hovertemplate="<b>%{x|%H:%M}</b><br>Failures: %{y:,.0f}<extra></extra>"
    ),
    row=2, col=1,
    secondary_y=True
)

fig3.update_xaxes(title_text="Time", row=2, col=1)
fig3.update_yaxes(title_text="Health Score (0-100)", row=1, col=1)
fig3.update_yaxes(title_text="Request Count", row=2, col=1)
fig3.update_yaxes(title_text="Failure Count", row=2, col=1, secondary_y=True)

fig3.update_layout(
    title={
        "text": "<b>📈 System Health Trend Analysis - Last 24 Hours</b>",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 22, "color": "#2c3e50"}
    },
    showlegend=True,
    height=800,
    width=1400,
    template="plotly_white",
    font={"family": "Arial, sans-serif", "size": 11},
    paper_bgcolor="#f8f9fa",
    plot_bgcolor="#ffffff",
    hovermode='x unified'
)

fig3.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')
fig3.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#ecf0f1')

fig3.show()

print("✅ Health trend analysis dashboard created!")

## 8. KQL Queries Reference

Below are all the KQL queries used to generate this dashboard. Copy and paste into Azure Application Insights to run directly:

### Query 1: Overall System Health Metrics

```kusto
requests
| where timestamp > ago(24h)
| summarize
    TotalRequests = count(),
    SuccessfulRequests = countif(success == true),
    FailedRequests = countif(success == false),
    AverageResponseTime = avg(duration),
    P95ResponseTime = percentile(duration, 95),
    P99ResponseTime = percentile(duration, 99)
| extend
    SuccessRate = (SuccessfulRequests * 100.0) / TotalRequests,
    FailureRate = (FailedRequests * 100.0) / TotalRequests
```

### Query 2: Exception and Error Analysis

```kusto
exceptions
| where timestamp > ago(24h)
| summarize 
    TotalExceptions = count(),
    CriticalCount = countif(severityLevel == 1),
    WarningCount = countif(severityLevel == 2),
    InfoCount = countif(severityLevel == 3)
| extend
    CriticalPercentage = (CriticalCount * 100.0) / TotalExceptions,
    WarningPercentage = (WarningCount * 100.0) / TotalExceptions
```

### Query 3: Performance by Operation

```kusto
requests
| where timestamp > ago(24h)
| summarize
    RequestCount = count(),
    AvgDuration = avg(duration),
    MaxDuration = max(duration),
    SuccessCount = countif(success == true)
    by name
| extend
    SuccessRate = (SuccessCount * 100.0) / RequestCount
| order by AvgDuration desc
| limit 10
```

### Query 4: Service Availability

```kusto
availabilityResults
| where timestamp > ago(24h)
| summarize
    TestCount = count(),
    PassedTests = countif(success == true),
    FailedTests = countif(success == false)
| extend
    AvailabilityPercentage = (PassedTests * 100.0) / TestCount
```

### Query 5: Request Timeline (Hourly Trend)

```kusto
requests
| where timestamp > ago(24h)
| summarize
    RequestCount = count(),
    SuccessCount = countif(success == true),
    FailureCount = countif(success == false),
    AvgResponse = avg(duration)
    by bin(timestamp, 1h)
| order by timestamp asc
```

### Query 6: Advanced - Request Performance Heatmap

```kusto
requests
| where timestamp > ago(24h)
| extend bin_time = bin(timestamp, 1h), duration_bucket = case(
    duration < 100, "< 100ms",
    duration < 300, "100-300ms",
    duration < 500, "300-500ms",
    duration < 1000, "500-1000ms",
    ">= 1000ms"
)
| summarize count() by bin_time, duration_bucket, success
| render columnchart
```

### Query 7: Error Analysis by Exception Type

```kusto
exceptions
| where timestamp > ago(24h)
| summarize
    ErrorCount = count(),
    AffectedUsers = dcount(user_Id),
    LastOccurrence = max(timestamp)
    by type, severityLevel
| order by ErrorCount desc
```

### Query 8: Custom Metric - System Health Score

```kusto
requests
| where timestamp > ago(24h)
| summarize
    SuccessRate = (countif(success == true) * 100.0) / count(),
    AvgResponse = avg(duration)
    by bin(timestamp, 1h)
| extend
    HealthScore = case(
        SuccessRate > 98 and AvgResponse < 300, 95,
        SuccessRate > 95 and AvgResponse < 500, 85,
        SuccessRate > 90 and AvgResponse < 1000, 70,
        SuccessRate > 85, 50,
        25
    )
| project timestamp, HealthScore, SuccessRate, AvgResponse
```

## 9. Summary & Insights

In [ ]:
# Generate comprehensive summary report
print("\n" + "="*70)
print("  🏥 AZURE APPLICATION INSIGHTS - SYSTEM HEALTH REPORT")
print("="*70)

print(f"\n📅 Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"📊 Analysis Period: Last 24 hours")

print(f"\n{'─'*70}")
print(f"  OVERALL HEALTH STATUS: {overall_health_score:.1f}/100 {health_status}")
print(f"{'─'*70}")

print(f"\n✅ PERFORMANCE METRICS:")
print(f"  • Total Requests: {int(health_data['TotalRequests']):,}")
print(f"  • Successful Requests: {int(health_data['SuccessfulRequests']):,}")
print(f"  • Failed Requests: {int(health_data['FailedRequests']):,}")
print(f"  • Average Response Time: {avg_response:.1f}ms")
print(f"  • P95 Response Time: {p95_response:.1f}ms")
print(f"  • P99 Response Time: {p99_response:.1f}ms")
print(f"  • Throughput: {throughput:.1f} requests/hour")

print(f"\n🎯 SUCCESS & AVAILABILITY:")
print(f"  • Success Rate: {success_rate:.2f}%")
print(f"  • Failure Rate: {failure_rate:.2f}%")
print(f"  • Service Availability: {availability:.2f}%")

print(f"\n❌ ERROR & EXCEPTION METRICS:")
print(f"  • Total Exceptions: {int(total_exceptions)}")
print(f"  • Critical Level Exceptions: {int(critical_exceptions)}")
print(f"  • Warning Level Exceptions: {int(warning_exceptions)}")
print(f"  • Info Level Messages: {int(exc_data['InfoCount'])}")

print(f"\n📈 TOP PERFORMING ENDPOINTS:")
for idx, row in df_performance.head(3).iterrows():
    print(f"  {idx+1}. {row['name']}")
    print(f"     ├─ Requests: {int(row['RequestCount']):,}")
    print(f"     ├─ Avg Response: {row['AvgDuration']:.0f}ms")
    print(f"     └─ Success Rate: {row['SuccessRate']:.2f}%")

print(f"\n⚠️  LOWEST PERFORMING ENDPOINTS:")
for idx, row in df_performance.tail(2).iterrows():
    status = "🔴" if row['SuccessRate'] < 90 else "🟡"
    print(f"  {status} {row['name']}")
    print(f"     ├─ Avg Response: {row['AvgDuration']:.0f}ms")
    print(f"     └─ Success Rate: {row['SuccessRate']:.2f}%")

print(f"\n💡 KEY INSIGHTS:")
if success_rate >= 98:
    print(f"  ✅ Excellent success rate! System is performing very well.")
elif success_rate >= 95:
    print(f"  🟢 Good success rate. System performance is stable.")
else:
    print(f"  🟡 Success rate below 95%. Investigate failures.")

if avg_response < 300:
    print(f"  ✅ Response times are excellent. System is responsive.")
elif avg_response < 500:
    print(f"  🟡 Response times are acceptable but could be optimized.")
else:
    print(f"  ⚠️  Response times are high. Consider optimization.")

if availability >= 99:
    print(f"  ✅ High availability! SLA targets are met.")
elif availability >= 95:
    print(f"  🟡 Availability is good but has room for improvement.")
else:
    print(f"  ⚠️  Availability is below SLA targets.")

if total_exceptions < 100:
    print(f"  ✅ Exception count is low. System is stable.")
elif total_exceptions < 200:
    print(f"  🟡 Moderate exception count. Monitor for patterns.")
else:
    print(f"  ⚠️  High exception count. Investigate root causes.")

print(f"\n📋 RECOMMENDATIONS:")
print(f"  • Monitor endpoints with response times > 1000ms")
print(f"  • Investigate exceptions with severity level = Critical")
print(f"  • Optimize endpoints with success rate < 95%")
print(f"  • Review performance trends in the hourly timeline")
print(f"  • Set up alerts for health score < 85")

print(f"\n" + "="*70)
print(f"  Report generated successfully!")
print(f"="*70 + "\n")